In [1]:
from glob import glob
import matplotlib.pyplot as plt
import fileinput
import collections
import re
from gensim.models import KeyedVectors, WordEmbeddingSimilarityIndex, TfidfModel
from gensim.similarities import MatrixSimilarity, SparseTermSimilarityMatrix, SoftCosineSimilarity
from gensim.corpora import Dictionary
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

partie = [
    'KO',
    'Konfederacja',
    'Lewica',
    'niez',
    'PiS',
    'PSL-Kukiz15'
]

In [2]:
stopwords = open("polish.stopwords.txt", encoding='utf-8').read().splitlines()

documents = []
tags = []

for p in partie:
    wypowiedzi_partii = ''
    for posl in glob('partie/' + p + '/*'):
        wypowiedzi = glob(posl + '/*')

        for wypowiedz in wypowiedzi:
            tags.append(wypowiedz[len('partie/'):-len('.txt')].replace('\\',': '))
            wyp_text = ''
            with fileinput.input(wypowiedz, openhook=fileinput.hook_encoded("windows-1250")) as wyp:
                for line in wyp:
                    wyp_text += line
            
            wyp_text = wyp_text.replace('\n', ' ')
            wyp_text = re.sub('[,.:;!?()]', '', wyp_text)
            documents.append(
                [word for word in wyp_text.lower().split() if word not in stopwords])

In [3]:
model = KeyedVectors.load("fasttext_100_3_polish.bin")
dictionary = Dictionary(documents)
tfidf = TfidfModel(dictionary=dictionary)

In [5]:
similarity_index = WordEmbeddingSimilarityIndex(model.wv)
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

In [6]:
index = SoftCosineSimilarity(tfidf[[dictionary.doc2bow(document) for document in documents]], similarity_matrix)

index.save('soft_cosine.index')

In [7]:
index = SoftCosineSimilarity.load('soft_cosine.index')

In [11]:
try:
    # query = input("Query: ").lower().split()
    query = "musimy stworzyć dużą ilość żłobków".lower().split()
    bow = dictionary.doc2bow(query, return_missing=True)
    query = tfidf[bow[0]]
    print("Słowa nierozpoznane: ", bow[1])
    similarities = index[query]
    for score, wyp in sorted(zip(similarities, tags), key= lambda t: t[0], reverse=True)[:20]:
        print('{:.3f} : {}'.format(score, wyp))
except Exception as e:
    print(e)

Słowa nierozpoznane:  {}
0.840 : PiS: Borowiak Joanna: 28-05-2020
0.780 : PiS: Wargocka Teresa: 28-05-2020
0.779 : PiS: Weber Rafał: 17-09-2020
0.764 : PiS: Morawiecki Mateusz: 06-04-2020
0.735 : Lewica: Czarzasty Włodzimierz: 08-01-2020
0.722 : KO: Gajewska Aleksandra: 28-05-2020
0.706 : KO: Chmiel Małgorzata: 27-10-2020
0.704 : PiS: Maląg Marlena Magdalena: 30-04-2020
0.704 : KO: Kołodziej Ewa: 12-02-2020
0.701 : PSL-Kukiz15: Krajewski Stefan: 17-09-2020
0.698 : KO: Budka Borys: 27-03-2020
0.689 : PiS: Lisiecki Paweł: 20-12-2019
0.684 : PSL-Kukiz15: Nowogórska Urszula: 16-09-2020
0.683 : PiS: Machałek Marzena: 17-09-2020
0.682 : PiS: Weber Rafał: 23-07-2020
0.680 : PiS: Piecha Bolesław: 21-10-2020
0.679 : Lewica: Krutul Paweł: 15-07-2020
0.673 : PSL-Kukiz15: Rzepa Jarosław: 28-05-2020
0.669 : PiS: Sipiera Zdzisław: 08-01-2020
0.669 : Lewica: Tomaszewski Tadeusz: 28-05-2020
